In [2]:

import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import numpy as np

In [17]:
perfumes = pd.read_csv(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\perfumes_batch1.csv")
perfumes

,Absolute URLs,Name,Brand,Ingredients,Classification,Family_and_Subfamily,Description
0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed..."
1,https://www.wikiparfum.com/en/fragrances/o-fem...,O FEMININ,M. Micallef,"Rose, Neroli, Musk, Pink Pepper, Patchouli, Le...",Floral Crisp Citrus,"FLORAL, CITRUS","Sensual, carnal and modern, this chic and bohe..."
2,https://www.wikiparfum.com/en/fragrances/miss-...,MISS DIOR PARFUM,Dior,"Cedarwood (Alaska), Patchouli, Mandarin, Straw...",Floral Crisp Fruity,"FLORAL, FRUITY","Between strength and grace, between recklessne..."
3,https://www.wikiparfum.com/en/fragrances/la-vi...,LA VIE EST BELLE ROSE EXTRAORDINAIRE,Lancome,"Rose, Musk, Woody Notes, Iris / Orris, Rose Da...",Floral Classical Rose,FLORAL,La Vie Est Belle Rose Extraordinaire is a rein...
4,https://www.wikiparfum.com/en/fragrances/karl-...,KARL LAGERFELD POUR FEMME ROUGE,Karl Lagerfeld,"Pear, Patchouli, Peach, Vanilla, Blackcurrant,...",Floral Amber Crisp Fruity,"AMBERY (ORIENTAL), FLORAL","Rouge is ""unique, modern and captivating,"" as ..."
...,...,...,...,...,...,...,...
10822,https://www.wikiparfum.com/en/fragrances/genet...,GENETIC BLISS,27 87,"Belambre (Woody ambery), Akigalawood (Patchoul...",Woods Classical,WOODY,"Genetic Bliss belongs to the collection ""Next ..."
10823,https://www.wikiparfum.com/en/fragrances/hamaca,HAMACA,27 87,"Coconut, Tonka Bean, Vanilla, Sandalwood, Marine",Amber Fresh Fruity,"AMBERY (ORIENTAL), FRUITY","A gentle breeze on salty skin, the soft warmth..."
10824,https://www.wikiparfum.com/en/fragrances/wande...,WANDERVOGEL,27 87,"Musk, Mint, Fennel Seeds, Shiso Leaves, Floral...",Soft Floral Fresh Green,"FLORAL, GREEN","It is the irresistible urge to get on a plane,..."
10825,https://www.wikiparfum.com/en/fragrances/elixi...,ELIXIR DE BOMBE,27 87,"Tangerine, Ginger, Hot Pepper, Raspberry, Ylan...",Dry Woods Fresh Fruity,"FRUITY, LEATHER","Boom, there it is, the right moment when you g..."


In [4]:
cols = []
for i in range(len(perfumes.columns)): 
    cols.append(perfumes.columns[i].lower().replace(' ', '_')) 
perfumes.columns = cols

In [5]:

perfumes.duplicated().sum()


0

In [6]:
perfumes.isna().sum()


absolute_urls            0
name                     8
brand                    8
ingredients              8
classification          10
family_and_subfamily     8
description             16
dtype: int64

In [7]:
perfumes = perfumes.dropna(subset=['name'])

In [8]:
perfumes.isna().sum()


absolute_urls           0
name                    0
brand                   0
ingredients             0
classification          2
family_and_subfamily    0
description             8
dtype: int64

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
def custom_tokenizer(text):
    return text.split(',')
count_vectorizer = CountVectorizer(tokenizer=custom_tokenizer)
bag_of_words = count_vectorizer.fit_transform(perfumes.ingredients)
bag_of_words.shape




(10819, 1457)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
import pickle, os, sys

similarity_matrix_sparse = cosine_similarity(bag_of_words, dense_output=False)
similarity_matrix_sparse.shape


(10819, 10819)

In [11]:
max_values = 5
num_rows = similarity_matrix_sparse.shape[0]

vect_index = []
vect_values = []

for index in range(num_rows):
    if index%5000 == 0 and index !=0: print(f'{index} calculated perfumes')
    if index+1 == num_rows: print(f'{index} calculated perfumes')
    perfume_search = similarity_matrix_sparse.getrow(index)
    top_similarity = np.argsort(perfume_search.data)[-max_values:][::-1]
    vect_index.append(perfume_search.indices[top_similarity])
    vect_values.append(perfume_search.data[top_similarity])

5000 calculated perfumes
10000 calculated perfumes
10818 calculated perfumes


In [12]:
# Assuming similarity_matrix_sparse is your sparse similarity matrix
# and perfumes is a DataFrame containing your product data, with columns 'name' and 'brand'

def find_similar_products(similarity_matrix_sparse, perfumes, query_product_id, max_values=5):
    # Get the similarity row corresponding to the query product
    query_product_similarity = similarity_matrix_sparse.getrow(query_product_id)
    
    # Get the top similar products indices
    top_similar_products = np.argsort(query_product_similarity.data)[-max_values-1:-1][::-1]  # Exclude the query product itself
    
    # Get the indices of similar products
    similar_product_ids = query_product_similarity.indices[top_similar_products]
    
    # Get the combined name and brand of similar products
    similar_product_names = [perfumes.loc[id, 'name'] + ' ' + perfumes.loc[id, 'brand'] for id in similar_product_ids]
    
    return similar_product_names

# Example usage:
# Define the product ID for which you want to find similar products
query_product_id = 980  # Replace 0 with the actual ID of the product

# Example usage:
# Find similar products
similar_products = find_similar_products(similarity_matrix_sparse, perfumes, query_product_id)
print("Products similar to", perfumes.loc[query_product_id, 'name'], "are:", similar_products)




Products similar to MAGENTA TANZANITE are: ['REVE DE COCO Adopt', 'SAVAGE JASMINE Sana Jardin', 'ALYSSA ASHLEY AMBRE GRIS Alyssa Ashley', 'SISTERLAND BLUE NEROLI Benetton', 'KEEMUN Welton London']


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

# Assuming 'perfumes' is your DataFrame containing perfume information
perfume_data = perfumes['ingredients']

# Drop any rows with missing values
perfumes = perfumes.dropna()

# Check if the DataFrame contains the 'name' and 'brand' columns
if 'name' not in perfumes.columns or 'brand' not in perfumes.columns:
    raise ValueError("DataFrame does not contain 'name' or 'brand' columns.")

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(perfume_data)

# Training the k-NN model
k = 5  # Number of neighbors
knn_model = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='cosine')
knn_model.fit(X)

# Initialize lists to store data
perfume_indices = []
similar_perfume_indices = []
rounded_similarity_scores = []

# Iterate through each perfume
for i in range(len(perfumes)):
    perfume_vector = X[i]

    # Query the k-NN model to find similar perfumes for the current perfume
    distances, indices = knn_model.kneighbors(perfume_vector.reshape(1, -1))  # Reshape for compatibility

    # Extract similar perfumes from the DataFrame
    similar_perfumes_indices = indices[0][1:]  # Exclude the first index, which is the current perfume itself
    similar_perfume_distances = distances[0][1:]  # Exclude the distance to the current perfume
    sorted_indices = np.argsort(similar_perfume_distances)  # Sort indices based on distance

    # Keep track of top three matches
    for j in sorted_indices[:3]:
        similar_perfume_index = similar_perfumes_indices[j]
        similarity_score = 1 - similar_perfume_distances[j]  # Cosine similarity
        rounded_similarity = round(similarity_score * 100)  # Rounded percentage match
        perfume_indices.append(i)
        similar_perfume_indices.append(similar_perfume_index)
        rounded_similarity_scores.append(rounded_similarity)

# Create DataFrame from lists
similar_perfumes_df = pd.DataFrame({
    'perfume_index': perfume_indices,
    'similar_perfume_index': similar_perfume_indices,
    'rounded_percentage_match': rounded_similarity_scores
})

# Save the DataFrame to a CSV file (optional)
similar_perfumes_df.to_csv('similar_perfumes_with_rounded_percentage_match.csv', index=False)

# Display the DataFrame
print(similar_perfumes_df)



       perfume_index  similar_perfume_index  rounded_percentage_match
0                  0                   4923                        51
1                  0                   7100                        45
2                  0                   5204                        45
3                  1                   6090                        77
4                  1                   5452                        71
...              ...                    ...                       ...
32422          10807                    224                        54
32423          10807                    395                        53
32424          10808                   3171                        66
32425          10808                   2952                        63
32426          10808                   1455                        59

[32427 rows x 3 columns]


In [15]:
result = similar_perfumes_df

In [19]:
result = pd.merge(result, perfumes, left_on='perfume_index', right_index=True, how='left')
result.to_csv('result1.csv')

In [31]:
result2 = pd.read_csv(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\result1.csv")


In [32]:
result2 = pd.merge(result2, perfumes, left_on='similar_perfume_index', right_index=True, how='left')
result2

,index,perfume_index,Absolute URLs_x,Name_x,Brand_x,Ingredients_x,Classification_x,Family_and_Subfamily_x,Description_x,similar_perfume_index,rounded_percentage_match,Absolute URLs,Name,Brand,Ingredients,Classification,Family_and_Subfamily,Description
0,0,0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed...",4923,51,https://www.wikiparfum.com/en/fragrances/betel...,BETELGEUSE,In Astra,"Coffee, Oakmoss, Osmanthus, Iris / Orris, Cori...",Soft Floral Crisp Iris,"FLORAL, ALDEHYDIC",Betelgeuse seduces those who can savour its tr...
1,1,0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed...",7100,45,https://www.wikiparfum.com/en/fragrances/legend-1,LEGEND,Michael Jordan,"Musk, Coffee, Patchouli, Lavender, Bergamot, C...",Woods Classical,WOODY,Legend by Michael Jordan is a masculine fragra...
2,2,0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Citruses, Dianthus, Benzoin, Bergamot, Incense...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed...",5204,45,https://www.wikiparfum.com/en/fragrances/ajar,AJAR,Jardins d'Ecrivains,"Pear, Oakmoss, Bergamot, Orchid, Jasmine, Ince...",Woods Fresh Fruity,"FRUITY, WOODY",Romain Gary achieved literary subterfuge when ...
3,3,1,https://www.wikiparfum.com/en/fragrances/o-fem...,O FEMININ,M. Micallef,"Rose, Neroli, Musk, Pink Pepper, Patchouli, Le...",Floral Crisp Citrus,"FLORAL, CITRUS","Sensual, carnal and modern, this chic and bohe...",6090,77,https://www.wikiparfum.com/en/fragrances/midni...,MIDNIGHT RAIN,La Prairie,"Musk, Pomegranate, Patchouli, Lily, Guava, Vet...",Woody Amber Classical,"AMBERY (ORIENTAL), WOODY","The embodiment of free-spirited passion, Midni..."
4,4,1,https://www.wikiparfum.com/en/fragrances/o-fem...,O FEMININ,M. Micallef,"Rose, Neroli, Musk, Pink Pepper, Patchouli, Le...",Floral Crisp Citrus,"FLORAL, CITRUS","Sensual, carnal and modern, this chic and bohe...",5452,71,https://www.wikiparfum.com/en/fragrances/sun-m...,SUN MEN SPORT,Jil Sander,"Violet Leaves, Cardamom, Bergamot, Cyclamen, A...",Aromatic Fougere Fresh Spicy,"SPICY, AROMATIC FOUGERE",The sun is the limit with Jil Sander Sun Men S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32422,32422,10807,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR PLATINUM,24,"Rose, Chamomile, Juniper Berries, Lemon, Incen...",Dry Woods Classical,"WOODY, LEATHER","As I wandered through the surreal garden, my t...",224,54,https://www.wikiparfum.com/en/fragrances/ibiza...,IBIZA BOHEMIA,Adopt,"Coconut, Tuberose, Skin Accord, Jasmine",Floral Amber Crisp White Flower,"AMBERY (ORIENTAL), FLORAL",Bring light and the bohemian spirit into your ...
32423,32423,10807,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR PLATINUM,24,"Rose, Chamomile, Juniper Berries, Lemon, Incen...",Dry Woods Classical,"WOODY, LEATHER","As I wandered through the surreal garden, my t...",395,53,https://www.wikiparfum.com/en/fragrances/n-10-...,No 10 WHITE OUD,Agonist,"Musk, Violet, Rose (Grasse), Patchouli, Ambrar...",Soft Floral Classical Musk,"MUSK SKIN, FLORAL","To celebrate their 10th anniversary ""No 10 Whi..."
32424,32424,10808,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR GOLD,24,"Guaiac Wood, Honey, Saffron, Cardamom, Cedarwo...",Woods Classical,WOODY,"I rose with the sun, carried by an imperceptib...",3171,66,https://www.wikiparfum.com/en/fragrances/only-...,ONLY THE BRAVE,Diesel,"Tangerine, Violet Leaves, Rosemary, Leaves, Le...",Woody Amber Fresh,"AMBERY (ORIENTAL), WOODY",It is a question of conviction. A question of ...
32425,32425,10808,https://www.wikiparfum.com/en/fragrances/24-el...,24 ELIXIR GOLD,24,"Gu

In [34]:
result2.to_csv('final_result.csv', index = False)
